__[jupyter magics](https://github.com/apowers313/roc/blob/master/experiments/2024.08.04-09.45.18-jupyter-magics/2024.08.04-09.45.18-jupyter-magics.ipynb)__

In [1]:
!date

Mon Aug  5 19:57:32 PDT 2024


# Breakpoint

In [2]:
# https://stackoverflow.com/questions/77580121/are-there-any-ways-to-actually-stop-awaited-long-running-asyncio-task

import asyncio

async def background_task(stop_event):
    while not stop_event.is_set():
        print('doing something')
        await asyncio.sleep(1)

async def main():
    stop_event = asyncio.Event()
    task = asyncio.create_task(background_task(stop_event))
    # Simulate running the task for some time
    await asyncio.sleep(5)
    stop_event.set() # Set the event to stop the background_task
    await task # Await the task to complete
    print('Done!')

# https://stackoverflow.com/questions/55409641/asyncio-run-cannot-be-called-from-a-running-event-loop-when-using-jupyter-no
# asyncio.run(main())
await main()

doing something
doing something
doing something
doing something
doing something
Done!


In [3]:
import asyncio

resume_event = asyncio.Event()

async def stop():
    while not resume_event.is_set():
        print("loopyboi")
        await asyncio.sleep(3)

def cont():
    resume_event.set() # Set the event to stop the background_task


def run_background():
    task = asyncio.create_task(stop())
    # Simulate running the task for some time
    await task # Await the task to complete
    print('Resuming!')

run_background()

SyntaxError: 'await' outside async function (1130455963.py, line 17)

In [ ]:
from threading import Thread, Lock
import time

brk_lock = Lock()
inbreak=False

def brk():
    global inbreak

    if inbreak:
        return
    print("breaking")
    inbreak=True
    brk_lock.acquire()

def cont():
    global inbreak

    if not inbreak:
        return
    print("continuing")
    inbreak=False
    brk_lock.release()

def thrd():
    waslocked=brk_lock.locked()
    while True:
        if brk_lock.locked():
            waslocked=True
            print("breaking.")
        with brk_lock:
            if waslocked:
                waslocked=False
                print("resuming...")
            print('Do some stuff')
        time.sleep(1)

t = Thread(target = thrd)
t.start()

Do some stuff


Do some stuff
Do some stuff
Do some stuff
Do some stuff
breaking.
resuming...
Do some stuff
Do some stuff
Do some stuff
Do some stuff
Do some stuff
Do some stuff
Do some stuff
breaking.
resuming...
Do some stuff
Do some stuff
Do some stuff
Do some stuff
breaking.


In [ ]:
brk()

breaking


In [ ]:
cont()

continuing


In [ ]:
brk()

# Magic

In [ ]:
# https://ipython.readthedocs.io/en/stable/config/custommagics.html
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_magic
def lmagic(line):
    "my line magic"
    return line

In [ ]:
%lmagic foobar

'foobar'

In [ ]:
# https://ipython.readthedocs.io/en/stable/config/custommagics.html
from __future__ import print_function
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

# The class MUST call this class decorator at creation time
@magics_class
class MyMagics(Magics):

    @line_magic
    def lmagic2(self, line):
        "my line magic"
        print("Full access to the main IPython object:", self.shell)
        print("Variables in the user namespace:", list(self.shell.user_ns.keys()))
        return line

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(MyMagics)

ip = get_ipython()
load_ipython_extension(ip)

In [ ]:
%lmagic2 asdf

Full access to the main IPython object: <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f94ef4cee90>
Variables in the user namespace: ['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'print_function', 'Magics', 'magics_class', 'line_magic', 'cell_magic', 'line_cell_magic', 'MyMagics', 'load_ipython_extension', '_i2', '_i3', '_i4', '_4', '_i5', 'ip', '_i6', '_6', '_i7', 'click', 'hello', 'args', '_i8', '_i9', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i20', '_i21', '_i22', '_i23', '_23', '_i24']


'asdf'

In [ ]:
lmagic2?

Docstring: my line magic
File:      /tmp/ipykernel_568538/3084839944.py

# Command Line Parsing

In [ ]:
import click

@click.command()
@click.option('--count', default=1, help='number of greetings')
@click.argument('name')
def hello(count, name):
    """This is a nifty little hello command."""
    for x in range(count):
        click.echo(f"Hello {name}!")

# standalone_mode makes it so that click doesn't call sys.exit() when it's done
hello("--count 3 Adam".split(), standalone_mode=False)

try:
    hello("--foo 3 John".split(), standalone_mode=False)
except Exception as e:
    print("ERROR:", e)

try:
    hello([], standalone_mode=False)
except Exception as e:
    print("ERROR:", e)

# for other arguments that can be passed to click:
# https://github.com/pallets/click/blob/main/src/click/core.py#L1238

hello(args=["--help"], prog_name="helpyboi", standalone_mode=False)

print("done.")

Hello Adam!
Hello Adam!
Hello Adam!
ERROR: No such option: --foo
ERROR: Missing parameter: name
Usage: helpyboi [OPTIONS] NAME

  This is a nifty little hello command.

Options:
  --count INTEGER  number of greetings
  --help           Show this message and exit.
done.


## Commands
- %roc \[n]
    - break on nth step
- %brk
    - list
    - remove
    - add
    - clear
- %cont
- %step \[n]
- %state \[module]
- %dump \[--format x] \[filename] 